In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, Dense, GRU, Input, BatchNormalization, Dropout
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-04-08 01:43:15.027318: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-08 01:43:15.031438: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-08 01:43:15.041090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744065795.059546 1144916 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744065795.064815 1144916 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 01:43:15.087784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Conv1D(filters=50, kernel_size=3, activation='relu', padding='causal'))

    model.add(LTC(AutoNCP(64, 32), return_sequences=True))
    model.add(LTC(AutoNCP(32, 16), return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.002, weight_decay=1e-7, use_ema=True), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [3]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = []
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0,0.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,0,0.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,0,0.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-04-08 01:43:20.151431: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 144, 50)        │         1,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 32)        │        29,540 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 16)             │         8,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,191 (153.09 KB)

 Trainable params: 39,191 (153.09 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5335 - auc: 0.4566 - loss: 0.6961 - val_accuracy: 0.5000 - val_auc: 0.8178 - val_loss: 0.6950
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5335 - auc: 0.6426 - loss: 0.6862 - val_accuracy: 0.5000 - val_auc: 0.7889 - val_loss: 0.6739
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5335 - auc: 0.7132 - loss: 0.6708 - val_accuracy: 0.7000 - val_auc: 0.7889 - val_loss: 0.6536
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6964 - auc: 0.7195 - loss: 0.6557 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.6223
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7615 - auc: 0.7190 - loss: 0.6336 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.5899
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7615 - auc: 0.7092 - loss: 0.6090 - val_accuracy: 0.8333 - val_auc: 0.8222 - val_loss: 0.5511
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy

1it [48:56, 2936.25s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.5494 - auc: 0.3803 - loss: 0.6912 - val_accuracy: 0.5000 - val_auc: 0.7867 - val_loss: 0.6839
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.5494 - auc: 0.7881 - loss: 0.6703 - val_accuracy: 0.5000 - val_auc: 0.7867 - val_loss: 0.6464
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.6603 - auc: 0.7916 - loss: 0.6321 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.5935
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.7667 - auc: 0.7873 - loss: 0.5844 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.5442
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.7667 - auc: 0.7908 - loss: 0.5443 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.5086
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.7667 - auc: 0.7822 - loss: 0.5173 - val_accuracy: 0.8000 - val_auc: 0.7867 - val_loss: 0.4850
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accurac

2it [1:41:58, 3081.20s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 61s 4s/step - accuracy: 0.5072 - auc: 0.4859 - loss: 0.6962 - val_accuracy: 0.7333 - val_auc: 0.8333 - val_loss: 0.6875
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.7821 - auc: 0.7844 - loss: 0.6838 - val_accuracy: 0.8333 - val_auc: 0.8556 - val_loss: 0.6728
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.7981 - auc: 0.7879 - loss: 0.6658 - val_accuracy: 0.8333 - val_auc: 0.8111 - val_loss: 0.6448
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.7956 - auc: 0.7904 - loss: 0.6354 - val_accuracy: 0.8333 - val_auc: 0.7667 - val_loss: 0.6089
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.8038 - auc: 0.7833 - loss: 0.5987 - val_accuracy: 0.8333 - val_auc: 0.7889 - val_loss: 0.5649
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.8038 - auc: 0.7869 - loss: 0.5572 - val_accuracy: 0.8333 - val_auc: 0.7400 - val_loss: 0.5141
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accurac

4it [3:36:57, 3364.62s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5346 - auc: 0.1802 - loss: 0.7032 - val_accuracy: 0.5000 - val_auc: 0.8000 - val_loss: 0.6925
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.5346 - auc: 0.7594 - loss: 0.6870 - val_accuracy: 0.6333 - val_auc: 0.8000 - val_loss: 0.6872
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8754 - auc: 0.8321 - loss: 0.6828 - val_accuracy: 0.8000 - val_auc: 0.7733 - val_loss: 0.6792
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.8754 - auc: 0.8343 - loss: 0.6710 - val_accuracy: 0.8000 - val_auc: 0.7733 - val_loss: 0.6658
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8754 - auc: 0.8343 - loss: 0.6523 - val_accuracy: 0.8000 - val_auc: 0.7733 - val_loss: 0.6470
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.8754 - auc: 0.8175 - loss: 0.6263 - val_accuracy: 0.8000 - val_auc: 0.7733 - val_loss: 0.6234
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

5it [4:21:29, 3137.86s/it]

CPU times: user 8h 40min 50s, sys: 2h 48min 8s, total: 11h 28min 58s
Wall time: 4h 21min 29s


In [ ]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

with open("./logs/Liquidv3_conv.txt", "w") as f:
    for i in range(NUM_EPOCHS):
        f.write(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}\n")
        f.write(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}\n")

Epoch 1: TRAIN Accuracy = 0.511 Loss = 0.699 AUC = 0.461
Epoch 1: VAL Accuracy = 0.56 Loss = 0.685 AUC = 0.805
Epoch 2: TRAIN Accuracy = 0.611 Loss = 0.675 AUC = 0.755
Epoch 2: VAL Accuracy = 0.647 Loss = 0.661 AUC = 0.804
Epoch 3: TRAIN Accuracy = 0.723 Loss = 0.65 AUC = 0.772
Epoch 3: VAL Accuracy = 0.78 Loss = 0.633 AUC = 0.79
Epoch 4: TRAIN Accuracy = 0.789 Loss = 0.621 AUC = 0.77
Epoch 4: VAL Accuracy = 0.813 Loss = 0.6 AUC = 0.792
Epoch 5: TRAIN Accuracy = 0.806 Loss = 0.59 AUC = 0.775
Epoch 5: VAL Accuracy = 0.82 Loss = 0.568 AUC = 0.796
Epoch 6: TRAIN Accuracy = 0.806 Loss = 0.559 AUC = 0.77
Epoch 6: VAL Accuracy = 0.82 Loss = 0.535 AUC = 0.787
Epoch 7: TRAIN Accuracy = 0.806 Loss = 0.53 AUC = 0.781
Epoch 7: VAL Accuracy = 0.82 Loss = 0.506 AUC = 0.781
Epoch 8: TRAIN Accuracy = 0.806 Loss = 0.505 AUC = 0.771
Epoch 8: VAL Accuracy = 0.82 Loss = 0.481 AUC = 0.784
Epoch 9: TRAIN Accuracy = 0.806 Loss = 0.485 AUC = 0.766
Epoch 9: VAL Accuracy = 0.82 Loss = 0.464 AUC = 0.803
Epoch 1

: 